In [1]:
import os
import sys
import pickle
from pyproj import CRS

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters
from lasso import metcouncil
from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import util

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# remote i/o

In [4]:
input_dir = os.path.join("D:/metcouncil_network_rebuild/data/processed/version_00")
lasso_dir = os.path.join("Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities")
output_dir = os.path.join("D:/metcouncil_network_rebuild/data/processed/version_01")

In [5]:
project_card_dir = os.path.join("D:/metcouncil_network_rebuild/ProjectCards_new_network/BaseCorrections")

In [6]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2022-04-17 00:13:09, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:13:09, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities


# Load Version00

In [7]:
link_file = os.path.join(input_dir, 'standard_networks', 'links.json')
node_file = os.path.join(input_dir, 'standard_networks', 'nodes.geojson')
shape_file = os.path.join(input_dir, 'standard_networks', 'shapes.geojson')

roadway_net = RoadwayNetwork.read(link_file = link_file, node_file = node_file, shape_file = shape_file)

2022-04-17 00:13:10, INFO: Reading from following files:
-D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\links.json
-D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\nodes.geojson
-D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\shapes.geojson.
2022-04-17 00:13:10, INFO: Reading from following files:
-D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\links.json
-D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\nodes.geojson
-D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\shapes.geojson.
2022-04-17 00:15:50, INFO: Read 1061566 links from D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\links.json
2022-04-17 00:15:50, INFO: Read 1061566 links from D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\links.json
2022-04-17 00:15:50, INFO: Read 414131 nodes from D:/metcouncil_network_rebuild/da

In [8]:
transit_net = TransitNetwork.read(feed_path = os.path.join(input_dir, 'standard_networks', 'transit'))

2022-04-17 00:17:13, INFO: Read in transit feed from: D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\transit
2022-04-17 00:17:13, INFO: Read in transit feed from: D:/metcouncil_network_rebuild/data/processed/version_00\standard_networks\transit
2022-04-17 00:17:13, INFO: Removing calendar.txt from transit network config because file not found
2022-04-17 00:17:13, INFO: Removing calendar.txt from transit network config because file not found
2022-04-17 00:17:13, INFO: Removing calendar_dates.txt from transit network config because file not found
2022-04-17 00:17:13, INFO: Removing calendar_dates.txt from transit network config because file not found
2022-04-17 00:17:14, INFO: Removing fare_attributes.txt from transit network config because file not found
2022-04-17 00:17:14, INFO: Removing fare_attributes.txt from transit network config because file not found
2022-04-17 00:17:14, INFO: Removing fare_rules.txt from transit network config because file not found


### Attribute the Network

In [9]:
roadway_net.links_df = roadway_net.links_df.drop('lanes', axis = 1)

In [10]:
roadway_net.links_df['bus_only'] = 0

In [11]:
roadway_net.links_df.drive_access.value_counts()

True     656673
False    404893
Name: drive_access, dtype: int64

In [12]:
r_net = metcouncil.calculate_number_of_lanes_from_reviewed_network(
    roadway_net=roadway_net,
    parameters=parameters,
)
r_net.links_df.lanes.value_counts()

2022-04-17 00:17:17, INFO: Calculating Number of Lanes as network variable: 'lanes'
2022-04-17 00:17:17, INFO: Calculating Number of Lanes as network variable: 'lanes'
2022-04-17 00:17:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:17:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:17:17, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2022-04-17 00:17:17, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2022-04-17 00:17:17, INFO: Finished calculating centroid connector variable: centroidconnect
2022-04-17 00:17:17, INFO: Finished calculating centroid connector variable: centroidconnect
2022-04-17 00:18:15, INFO: Finished calculating number of lanes to: lanes
2022-04-17 00:18:15, INFO: Finished calculating number of lanes to: lanes


1.0    1026505
2.0      32020
3.0       2595
4.0        369
5.0         71
6.0          5
7.0          1
Name: lanes, dtype: int64

In [13]:
r_net = metcouncil.calculate_assign_group_and_roadway_class_from_reviewed_network(
        roadway_net=r_net,
        parameters=parameters,
)
r_net.links_df.assign_group.value_counts()

2022-04-17 00:18:16, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2022-04-17 00:18:16, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2022-04-17 00:18:16, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2022-04-17 00:18:16, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2022-04-17 00:20:15, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class
2022-04-17 00:20:15, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class


101.0    404893
50.0     294101
7.0      101584
6.0       76802
5.0       15025
15.0       3045
3.0        2590
1.0        2455
4.0        2036
2.0        1258
11.0        153
Name: assign_group, dtype: int64

In [14]:
r_net.links_df.roadway_class.value_counts()

101.0    404893
50.0     304024
40.0     124841
30.0      42279
20.0      20590
60.0       5416
10.0       1780
70.0        119
Name: roadway_class, dtype: int64

### Add Rail links and nodes

In [15]:
r_net = metcouncil.add_rail_links_and_nodes(
    roadway_network = r_net,
    parameters = parameters,
    rail_links_file = os.path.join(input_dir, 'standard_networks', 'rail_links.geojson'),
    rail_nodes_file = os.path.join(input_dir, 'standard_networks', 'rail_nodes.geojson'),
)

2022-04-17 00:20:16, INFO: Adding centroid and centroid connector to standard network
2022-04-17 00:20:16, INFO: Adding centroid and centroid connector to standard network
2022-04-17 00:20:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:20:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:20:18, INFO: Finished adding rail links and nodes connectors
2022-04-17 00:20:18, INFO: Finished adding rail links and nodes connectors


In [16]:
# check if missing IDs
# rail nodes does not have osm IDs, they have shst IDs
# rail links does not have osm IDs, they have shst IDs

# if node missing shst id
print(r_net.nodes_df.shst_node_id.isnull().sum())
print(r_net.nodes_df.shst_node_id.nunique())
# if node missing osm id
print(r_net.nodes_df.osm_node_id.isnull().sum())
# if node missing osm id
print(r_net.nodes_df.osm_node_id.dtype)
print(r_net.nodes_df[r_net.nodes_df.osm_node_id == 0])
print(r_net.nodes_df.osm_node_id.nunique())
# if node missing model node id
print(r_net.nodes_df.model_node_id.nunique())

# if link missing 
print(r_net.links_df.shstReferenceId.isnull().sum())
print(r_net.links_df.shstReferenceId.nunique())
print(r_net.links_df.model_link_id.nunique())

# if link missing node id
print(r_net.links_df.fromIntersectionId.isnull().sum())
print(r_net.links_df.toIntersectionId.isnull().sum())
print(r_net.links_df.u.isnull().sum())
print(r_net.links_df[r_net.links_df.u == 0])
print(r_net.links_df.v.isnull().sum())
print(r_net.links_df[r_net.links_df.v == 0])

0
414222
1995
float64
Empty GeoDataFrame
Columns: [osm_node_id, shst_node_id, county, drive_access, walk_access, bike_access, model_node_id, rail_only, geometry, X, Y]
Index: []
412227
414222
0
1061656
1061656
0
0
5081
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, ref, name, highway, service, width, maxspeed, access, junction, bridge, tunnel, landuse, area, key, forward, backReferenceId, metadata, source, roadway, drive_access, walk_access, bike_access, county, length, A, B, model_link_id, locationReferences, rail_only, geometry, bus_only, centroidconnect, lanes, assign_group, roadway_class]
Index: []

[0 rows x 48 columns]
5081
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, ref, name, highway, service, width, maxspeed, access, junction, bri

# Create Scenario 00

In [17]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}

In [18]:
version_00_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2022-04-17 00:20:23, INFO: Creating Scenario
2022-04-17 00:20:23, INFO: Creating Scenario


In [19]:
working_scenario_filename = os.path.join(output_dir, 'version_00_standard_network.pickle')
pickle.dump(version_00_scenario, open(working_scenario_filename, 'wb'))

In [20]:
r_net.links_df.shape

(1061656, 48)

In [21]:
r_net.links_df.model_link_id.max()

1061656

# Create Scenario 01

In [22]:
version_01_scenario = Scenario.create_scenario(
    base_scenario=version_00_scenario,
    card_directory = project_card_dir,
    validate_project_cards=False
)

2022-04-17 00:21:37, INFO: Creating Scenario
2022-04-17 00:21:37, INFO: Creating Scenario


z:\data\users\sijia\met_council\github\network_wrangler\network_wrangler\network_wrangler\projectcard.py:114: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [23]:
version_01_scenario.apply_all_projects()

2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group and roadway class
2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group and roadway class
2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group and roadway class
2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group and roadway class
2022-04-17 00:21:41, INFO: Applying Project to Roadway Network: correct year 2018 assignment group and roadway class
2022-04-17 00:21:41, INFO: Applying Project to Roadway Network: correct year 2018 assignment group and roadway class
2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:41, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:41, INFO: Applying Project to Roadway Network: correct year 2018 assignment group
2022-04-17 00:21:41, INFO: 

C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-04-17 00:21:46, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:46, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:46, INFO: Applying Project to Roadway Network: correct year 2018 assignment group
2022-04-17 00:21:46, INFO: Applying Project to Roadway Network: correct year 2018 assignment group


C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-04-17 00:21:52, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:52, INFO: Applying correct year 2018 assignment group
2022-04-17 00:21:52, INFO: Applying Project to Roadway Network: correct year 2018 assignment group
2022-04-17 00:21:52, INFO: Applying Project to Roadway Network: correct year 2018 assignment group


...

C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-04-17 00:37:22, INFO: Applying network cleanup 1 asgngrp 15
2022-04-17 00:37:22, INFO: Applying network cleanup 1 asgngrp 15
2022-04-17 00:37:22, INFO: Applying network cleanup 1 asgngrp 15
2022-04-17 00:37:22, INFO: Applying network cleanup 1 asgngrp 15
2022-04-17 00:37:22, INFO: Applying Project to Roadway Network: network cleanup 1 asgngrp 15
2022-04-17 00:37:22, INFO: Applying Project to Roadway Network: network cleanup 1 asgngrp 15


C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-04-17 00:37:24, INFO: Applying network cleanup 1 asgngrp 1
2022-04-17 00:37:24, INFO: Applying network cleanup 1 asgngrp 1
2022-04-17 00:37:24, INFO: Applying network cleanup 1 asgngrp 1
2022-04-17 00:37:24, INFO: Applying network cleanup 1 asgngrp 1
2022-04-17 00:37:24, INFO: Applying Project to Roadway Network: network cleanup 1 asgngrp 1
2022-04-17 00:37:24, INFO: Applying Project to Roadway Network: network cleanup 1 asgngrp 1


C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [24]:
version_01_scenario.applied_projects

['correct year 2018 assignment group and roadway class',
 'correct year 2018 assignment group',
 'network cleanup remove link',
 'Manual Changes to Assignment Group and Roadway Class',
 'network cleanup 1 lanes 7',
 'network cleanup 1 lanes 6',
 'network cleanup 1 lanes 5',
 'network cleanup 1 lanes 4',
 'network cleanup 1 lanes 3',
 'network cleanup 1 lanes 2e',
 'network cleanup 1 lanes 2d part 2',
 'network cleanup 1 lanes 2d',
 'network cleanup 1 lanes 2c part 2',
 'network cleanup 1 lanes 2c',
 'network cleanup 1 lanes 2b',
 'network cleanup 1 lanes 2a',
 'network cleanup 1 lanes 1 h',
 'network cleanup 1 lanes 1 g',
 'network cleanup 1 lanes 1 f part 2',
 'network cleanup 1 lanes 1 f',
 'network cleanup 1 lanes 1 e part 2',
 'network cleanup 1 lanes 1 e',
 'network cleanup 1 lanes 1 d part 2',
 'network cleanup 1 lanes 1 d',
 'network cleanup 1 lanes 1 c part 2',
 'network cleanup 1 lanes 1 c',
 'network cleanup 1 lanes 1 b part 2',
 'network cleanup 1 lanes 1 b',
 'network clean

In [25]:
version_01_scenario.road_net.links_df.shape

(1061718, 50)

In [26]:
version_01_scenario.road_net.links_df.model_link_id.nunique()

1061718

In [27]:
version_01_scenario.road_net.nodes_df.shape

(414253, 11)

In [28]:
version_01_scenario.road_net.nodes_df.model_node_id.nunique()

414253

In [29]:
version_01_scenario.road_net.links_df['access'] = version_01_scenario.road_net.links_df['access'].fillna('')
version_01_scenario.road_net.links_df['access'] = version_01_scenario.road_net.links_df['access'].apply(
    lambda x: util.shorten_name(x)
)

In [30]:
version_01_scenario.road_net.links_df['name'] = version_01_scenario.road_net.links_df['name'].fillna('')
version_01_scenario.road_net.links_df['name'] = version_01_scenario.road_net.links_df['name'].apply(
    lambda x: util.shorten_name(x)
)

# Save version 01 standard networks

In [32]:
version_01_scenario.road_net.write(path = os.path.join(output_dir, 'standard_networks'))

# Save version 01 standard network as pickle file

In [33]:
working_scenario_filename = os.path.join(output_dir, 'version_01_standard_network.pickle')
pickle.dump(version_01_scenario, open(working_scenario_filename, 'wb'))

# Make Travel Model Network

In [34]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = version_01_scenario.road_net, 
    parameters = parameters
)

2022-04-17 00:52:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:52:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities


### Add centroid and centroid connectors

In [35]:
m_net = metcouncil.add_centroid_and_centroid_connector(
    roadway_network = m_net,
    parameters = parameters,
    centroid_file = os.path.join(input_dir, 'standard_networks', 'centroid_node.pickle'),
    centroid_connector_link_file = os.path.join(input_dir, 'standard_networks', 'cc_link.pickle'),
    centroid_connector_shape_file = os.path.join(input_dir, 'standard_networks', 'cc_shape.pickle'),
)

2022-04-17 00:52:12, INFO: Adding centroid and centroid connector to standard network
2022-04-17 00:52:12, INFO: Adding centroid and centroid connector to standard network
2022-04-17 00:52:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:52:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 00:52:16, INFO: Finished adding centroid and centroid connectors
2022-04-17 00:52:16, INFO: Finished adding centroid and centroid connectors


### Add Rail access and egress links

In [36]:
m_net = metcouncil.add_rail_ae_connections(
    m_net,
    parameters
)

2022-04-17 00:52:16, INFO: Creating rail access and egress connection links
2022-04-17 00:52:16, INFO: Creating rail access and egress connection links


C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_wsp\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [37]:
m_net.roadway_standard_to_met_council_network()

2022-04-17 00:52:37, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2022-04-17 00:52:37, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2022-04-17 00:52:37, INFO: Didn't detect managed lanes in network.
2022-04-17 00:52:37, INFO: Didn't detect managed lanes in network.
2022-04-17 00:52:37, INFO: Creating calculated roadway variables.
2022-04-17 00:52:37, INFO: Creating calculated roadway variables.
2022-04-17 00:52:37, INFO: Area Type Variable 'area_type' already in network. Returning without overwriting.
2022-04-17 00:52:37, INFO: Area Type Variable 'area_type' already in network. Returning without overwriting.
2022-04-17 00:52:37, INFO: Overwriting existing County Variable 'county' already in network
2022-04-17 00:52:37, INFO: Overwriting existing County Variable 'county' already in network
2022-04-17 00:52:37, INFO: Adding roadway network variable for county using a spatial join with: Z:/Data

z:\data\users\sijia\met_council\github\client_met_council_wrangler_utilities\lasso\roadway.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid
z:\data\users\sijia\met_council\github\client_met_council_wrangler_utilities\lasso\roadway.py:1225: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  self.create_calculated_variables()
z:\data\users\sijia\met_council\github\client_met_council_wrangler_utilities\lasso\roadway.py:292: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  joined_gdf = gpd.sjoin(centroids_gdf, county_gdf, how=

2022-04-17 00:55:35, INFO: Finished Calculating county variable: county
2022-04-17 00:55:35, INFO: Finished Calculating county variable: county
2022-04-17 00:55:40, INFO: Calculating MPO as roadway network variable: mpo
2022-04-17 00:55:40, INFO: Calculating MPO as roadway network variable: mpo
2022-04-17 00:55:40, INFO: Finished calculating MPO variable: mpo
2022-04-17 00:55:40, INFO: Finished calculating MPO variable: mpo
2022-04-17 00:55:40, INFO: Adding Counts
2022-04-17 00:55:40, INFO: Adding Counts
2022-04-17 00:55:40, INFO: Adding Variable AADT using Shared Streets Reference from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2022-04-17 00:55:40, INFO: Adding Variable AADT using Shared Streets Reference from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2022-04-17 00:55:58, INFO: Added variable: AADT using Shared S

2022-04-17 00:57:36, WARNING: Specified variable to split: price not in network variables: Index(['shstReferenceId', 'id', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'u', 'v', 'nodeIds', 'wayId', 'roadClass', 'oneWay',
       'roundabout', 'link', 'oneway', 'ref', 'name', 'highway', 'service',
       'width', 'maxspeed', 'access', 'junction', 'bridge', 'tunnel',
       'landuse', 'area', 'key', 'forward', 'backReferenceId', 'metadata',
       'source', 'roadway', 'drive_access', 'walk_access', 'bike_access',
       'county', 'length', 'A', 'B', 'model_link_id', 'locationReferences',
       'rail_only', 'geometry', 'bus_only', 'centroidconnect', 'lanes',
       'assign_group', 'roadway_class', 'area_type', 'ramp_flag', 'mpo',
       'AADT', 'count_AM', 'count_MD', 'count_PM', 'count_NT', 'count_daily',
       'count_year', 'segment_id', 'managed', 'distance', 'trn_priority_AM',
       'trn_priority_MD', 'trn_priority_PM', 'trn_priority_NT',
       'ttime_assert_A

In [38]:
# check if missing IDs
# centroids does not have osm and shst IDs
# centroid connectors does not have osm and shst IDs

# if node missing shst id
print(m_net.nodes_df.shst_node_id.isnull().sum())
print(m_net.nodes_df.shst_node_id.nunique())

# if node missing model node id
print(m_net.nodes_df.model_node_id.nunique())

# if link missing 
print(m_net.links_df.shstReferenceId.isnull().sum())
print(m_net.links_df.shstReferenceId.nunique())
print(m_net.links_df.model_link_id.nunique())

# if link missing node id
print(m_net.links_df.fromIntersectionId.isnull().sum())
print(m_net.links_df.toIntersectionId.isnull().sum())

0
414223
417283
0
1061657
1109542
0
0


In [39]:
m_net.nodes_metcouncil_df.columns

Index(['osm_node_id', 'shst_node_id', 'drive_access', 'walk_access',
       'bike_access', 'N', 'rail_only', 'geometry', 'X', 'Y', 'county'],
      dtype='object')

In [40]:
m_net.links_metcouncil_df.columns

Index(['shstReferenceId', 'id', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'u', 'v', 'nodeIds', 'wayId', 'roadClass', 'oneWay',
       'roundabout', 'link', 'oneway', 'ref', 'name', 'highway', 'service',
       'width', 'maxspeed', 'access', 'junction', 'bridge', 'tunnel',
       'landuse', 'area', 'key', 'forward', 'backReferenceId', 'metadata',
       'source', 'roadway', 'drive_access', 'walk_access', 'bike_access',
       'county', 'length', 'A', 'B', 'model_link_id', 'locationReferences',
       'rail_only', 'bus_only', 'centroidconnect', 'lanes', 'assign_group',
       'roadway_class', 'area_type', 'ramp_flag', 'mpo', 'AADT', 'count_AM',
       'count_MD', 'count_PM', 'count_NT', 'count_daily', 'count_year',
       'segment_id', 'managed', 'distance', 'trn_priority_AM',
       'trn_priority_MD', 'trn_priority_PM', 'trn_priority_NT',
       'ttime_assert_AM', 'ttime_assert_MD', 'ttime_assert_PM',
       'ttime_assert_NT', 'lanes_AM', 'lanes_MD', 'lanes_PM',

# Write model network as shapefile

In [41]:
out_cols = ['model_link_id', 'id', 'assign_group', 'drive_access', 'roadway_class',
            'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT', 'segment_id', 'HOV', 
            'price_sov_AM', 'geometry', 'managed']

m_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'model_networks', 'shapefiles', 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'model_networks', 'shapefiles', 'nodes.shp'),
    link_output_variables = out_cols,
    data_to_csv = False,
    data_to_dbf = True,
    export_drive_only = False, # if user only wants drive links/nodes in the shapefile
)

2022-04-17 00:59:28, INFO: Writing Network as Shapefile
2022-04-17 00:59:28, INFO: Writing Network as Shapefile
2022-04-17 00:59:28, INFO: Renaming DBF Node Variables
2022-04-17 00:59:28, INFO: Renaming DBF Node Variables
2022-04-17 00:59:28, INFO: Renaming variables so that they are DBF-safe
2022-04-17 00:59:28, INFO: Renaming variables so that they are DBF-safe
2022-04-17 00:59:38, INFO: Renaming DBF Link Variables
2022-04-17 00:59:38, INFO: Renaming DBF Link Variables
2022-04-17 00:59:38, INFO: Renaming variables so that they are DBF-safe
2022-04-17 00:59:38, INFO: Renaming variables so that they are DBF-safe
2022-04-17 00:59:39, INFO: Writing Node Shapes:
 - D:/metcouncil_network_rebuild/data/processed/version_01\model_networks\shapefiles\nodes.shp
2022-04-17 00:59:39, INFO: Writing Node Shapes:
 - D:/metcouncil_network_rebuild/data/processed/version_01\model_networks\shapefiles\nodes.shp
2022-04-17 01:00:34, INFO: Writing Link Shapes:
 - D:/metcouncil_network_rebuild/data/processe

# Write model network for Cube

In [42]:
m_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'model_networks', 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'model_networks', 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'model_networks', 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'model_networks', 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'model_networks', 'make_complete_network_from_fixed_width_file.s'),
)

2022-04-17 01:03:42, INFO: Starting fixed width conversion
2022-04-17 01:03:42, INFO: Starting fixed width conversion
2022-04-17 01:15:30, INFO: Writing out link database
2022-04-17 01:15:30, INFO: Writing out link database
2022-04-17 01:15:46, INFO: Writing out link header and width ----
2022-04-17 01:15:46, INFO: Writing out link header and width ----
2022-04-17 01:15:46, INFO: Starting fixed width conversion
2022-04-17 01:15:46, INFO: Starting fixed width conversion
2022-04-17 01:16:38, INFO: Writing out node database
2022-04-17 01:16:38, INFO: Writing out node database
2022-04-17 01:16:39, INFO: Writing out node header and width
2022-04-17 01:16:39, INFO: Writing out node header and width


In [43]:
version_01_scenario.transit_net.set_roadnet(version_01_scenario.road_net, validate_consistency=False)

In [44]:
standard_transit_net = StandardTransit.fromTransitNetwork(version_01_scenario.transit_net, parameters=parameters)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, 'model_networks', "transit.lin"))

2022-04-17 01:19:23, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 01:19:23, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-04-17 01:19:23, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard
2022-04-17 01:19:23, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard
